# Testing Neural Network and Cross Validation

In [1]:
# Utils class with functions for model development, testing and evaluation
import utils as u
%matplotlib inline

In [2]:
df_first_run = u.pd.read_csv('./Model Data Set STDEV MA (pseudo).csv',sep=',')
df_first_run.head()

,Unnamed: 0,CREDIT_UNION_ID,ACCOUNT_NUM,FLG_202201,FLG_202202,FLG_202203,FLG_202204,FLG_202205,FLG_202206,DEEP_CHANNELS_202201,...,PAYMENTS_MA_3,PAYMENTS_MA_VAR,AMP_MA_1,AMP_MA_2,AMP_MA_3,AMP_MA_VAR,NUM_TRANSACTIONS_MA_1,NUM_TRANSACTIONS_MA_2,NUM_TRANSACTIONS_MA_3,NUM_TRANSACTIONS_MA_VAR
0,0,A,ZWZZ!W,1,1,1,1,1,1,0,...,8.666667,2.666667,3.666667,3.333333,2.666667,-1.000000,94.666667,105.333333,99.333333,4.666667
1,1,A,&WXYY&,1,1,1,1,1,1,0,...,0.000000,-2.000000,2.000000,1.666667,1.666667,-0.333333,8.666667,12.000000,13.333333,4.666667
2,2,A,Y%@YZ&,1,1,1,1,1,1,0,...,0.000000,0.000000,2.000000,2.000000,1.666667,-0.333333,9.333333,9.333333,11.333333,2.000000
3,3,A,!W%&#!,1,1,1,1,1,1,0,...,1.333333,0.000000,2.666667,3.000000,2.666667,0.000000,32.666667,38.000000,54.000000,21.333333
4,4,A,%##AXY,1,1,1,1,1,1,0,...,2.666667,-0.666667,3.000000,2.666667,2.666667,-0.333333,16.000000,10.666667,8.666667,-7.333333


In [3]:
df_big_blind_test = u.pd.read_csv('./Big Blind Predict Test.csv',sep=',')
df_big_blind_test.head()

,Unnamed: 0,FLG_202201,FLG_202202,FLG_202203,FLG_202204,FLG_202205,FLG_202206,DEEP_CHANNELS_202201,DEEP_CHANNELS_202202,DEEP_CHANNELS_202203,...,PAYMENTS_MA_3,PAYMENTS_MA_VAR,AMP_MA_1,AMP_MA_2,AMP_MA_3,AMP_MA_VAR,NUM_TRANSACTIONS_MA_1,NUM_TRANSACTIONS_MA_2,NUM_TRANSACTIONS_MA_3,NUM_TRANSACTIONS_MA_VAR
0,0,1,1,1,1,1,1,0,0,0,...,10.666667,2.666667,2.333333,2.666667,3.000000,0.666667,26.666667,29.333333,32.000000,5.333333
1,1,1,1,1,1,1,1,0,0,0,...,3.333333,2.000000,2.000000,2.000000,2.000000,0.000000,18.000000,18.666667,19.333333,1.333333
2,2,1,1,1,1,1,1,0,0,0,...,15.333333,5.333333,2.333333,2.666667,2.666667,0.333333,53.333333,72.000000,79.333333,26.000000
3,3,1,1,1,1,1,1,0,0,0,...,1.333333,1.333333,2.000000,2.000000,1.666667,-0.333333,34.000000,20.666667,19.333333,-14.666667
4,4,1,1,1,1,1,1,0,0,0,...,4.000000,1.333333,3.000000,3.000000,3.333333,0.333333,25.333333,29.333333,38.666667,13.333333


In [4]:
# We make sure to create a copy of the data before we start altering it. Note that we don't change the original data we loaded.
data_first_run = df_first_run.copy(deep=False)
data_big_blind_test = df_big_blind_test.copy(deep=False)

# Lowering Dataset complexity

In [5]:
data_first_run = u.pd.DataFrame()
data_first_run['FLG_INNACTIVITY'] = df_first_run['FLG_202206']
data_first_run.loc[data_first_run['FLG_INNACTIVITY'] == 1] = 2
data_first_run.loc[data_first_run['FLG_INNACTIVITY'] == 0] = 1
data_first_run.loc[data_first_run['FLG_INNACTIVITY'] == 2] = 0
data_first_run['NUM_TRANSACTIONS_MA_1'] = df_first_run['NUM_TRANSACTIONS_MA_1']
data_first_run['NUM_TRANSACTIONS_MA_2'] = df_first_run['NUM_TRANSACTIONS_MA_2']
data_first_run['NUM_TRANSACTIONS_MA_3'] = df_first_run['NUM_TRANSACTIONS_MA_3']
data_first_run['NUM_TRANSACTIONS_MA_VAR'] = df_first_run['NUM_TRANSACTIONS_MA_VAR']
data_first_run

,FLG_INNACTIVITY,NUM_TRANSACTIONS_MA_1,NUM_TRANSACTIONS_MA_2,NUM_TRANSACTIONS_MA_3,NUM_TRANSACTIONS_MA_VAR
0,0,94.666667,105.333333,99.333333,4.666667
1,0,8.666667,12.000000,13.333333,4.666667
2,0,9.333333,9.333333,11.333333,2.000000
3,0,32.666667,38.000000,54.000000,21.333333
4,0,16.000000,10.666667,8.666667,-7.333333
...,...,...,...,...,...
91843,0,76.666667,79.333333,102.666667,26.000000
91844,0,16.000000,18.666667,18.000000,2.000000
91845,0,3.000000,2.666667,2.666667,-0.333333
91846,0,15.333333,8.000000,8.666667,-6.666667


In [6]:
data_big_blind_test = u.pd.DataFrame()
data_big_blind_test['FLG_INNACTIVITY'] = df_first_run['FLG_202206']
data_big_blind_test.loc[data_big_blind_test['FLG_INNACTIVITY'] == 1] = 2
data_big_blind_test.loc[data_big_blind_test['FLG_INNACTIVITY'] == 0] = 1
data_big_blind_test.loc[data_big_blind_test['FLG_INNACTIVITY'] == 2] = 0
data_big_blind_test['NUM_TRANSACTIONS_MA_1'] = df_first_run['NUM_TRANSACTIONS_MA_1']
data_big_blind_test['NUM_TRANSACTIONS_MA_2'] = df_first_run['NUM_TRANSACTIONS_MA_2']
data_big_blind_test['NUM_TRANSACTIONS_MA_3'] = df_first_run['NUM_TRANSACTIONS_MA_3']
data_big_blind_test['NUM_TRANSACTIONS_MA_VAR'] = df_first_run['NUM_TRANSACTIONS_MA_VAR']
data_big_blind_test

,FLG_INNACTIVITY,NUM_TRANSACTIONS_MA_1,NUM_TRANSACTIONS_MA_2,NUM_TRANSACTIONS_MA_3,NUM_TRANSACTIONS_MA_VAR
0,0,94.666667,105.333333,99.333333,4.666667
1,0,8.666667,12.000000,13.333333,4.666667
2,0,9.333333,9.333333,11.333333,2.000000
3,0,32.666667,38.000000,54.000000,21.333333
4,0,16.000000,10.666667,8.666667,-7.333333
...,...,...,...,...,...
91843,0,76.666667,79.333333,102.666667,26.000000
91844,0,16.000000,18.666667,18.000000,2.000000
91845,0,3.000000,2.666667,2.666667,-0.333333
91846,0,15.333333,8.000000,8.666667,-6.666667


# Declaring Global Variables

In [53]:
#Declare independent variables (X) and dependent variable (y) for the Model Training and Setup dataset
X = data_first_run.drop(columns=['FLG_INNACTIVITY']) # Drop prediction column
y = data_first_run['FLG_INNACTIVITY']

#Declare independent variables (X) and dependent variable (y) for the Big Blind Predict Test dataset
X_bbt = data_big_blind_test.drop(columns=['FLG_INNACTIVITY']) # Drop prediction column
y_bbt = data_big_blind_test['FLG_INNACTIVITY']

#set shared model, scaler and splitter variables
random_state = 13
test_size = 0.30
verbose = 'off'
print_report = 'off'

#set model names
models = [
          'Random Forest',
          'Logistic Regression',
         ]

#set resampling method names
resamplers = [
              'Baseline',
              'Random Over Sampling',
              'SMOTE',
              'Near Miss KNN',
              'Random Under Sampling',
             ]

# Neural Network specific Libs

Implementation References:
https://www.pluralsight.com/guides/machine-learning-neural-networks-scikit-learn

In [59]:
#Neural Network Test

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_curve, accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from sklearn.metrics import get_scorer_names
#get_scorer_names() -- Check scorrer names to pass during cross validation

# Neural Network Global Instances

In [38]:
# Baseline Model
mlp = MLPClassifier(random_state = random_state)

# Scaling
scaler = preprocessing.MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled
        
# Split train test data
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = u.train_test_split(X_scaled,y,test_size = test_size,random_state = random_state)        

# Baseline testing

In [50]:
# Fiting
mlp_local = mlp.fit(X_train,y_train)

# Predicing
predict_train = mlp_local.predict(X_train)
predict_test = mlp_local.predict(X_test)

#Evaluating
print('Results on Train data')
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train,zero_division=0))

print('Results on Test data')
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test,zero_division=0))

Results on Train data
[[62312     8]
 [ 1971     2]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     62320
           1       0.20      0.00      0.00      1973

    accuracy                           0.97     64293
   macro avg       0.58      0.50      0.49     64293
weighted avg       0.95      0.97      0.95     64293

Results on Test data
[[26658     2]
 [  895     0]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     26660
           1       0.00      0.00      0.00       895

    accuracy                           0.97     27555
   macro avg       0.48      0.50      0.49     27555
weighted avg       0.94      0.97      0.95     27555



# Scaling and Resampling Testing on Baseline model

## StandardScaler and RandomOverSampler

In [45]:
# Resampling
resampler = RandomOverSampler(random_state=random_state)
X_train_resampled, y_train_resampled= resampler.fit_resample(X_train, y_train)

# Fitting
mlp_local = mlp.fit(X_train_resampled,y_train_resampled)

#Predicting
predict_train_resampled = mlp_local.predict(X_train_resampled)
predict_test_resampled = mlp_local.predict(X_test)

#Evaluating
print('Results on Train data')
print(confusion_matrix(y_train_resampled,predict_train_resampled))
print(classification_report(y_train_resampled,predict_train_resampled,zero_division=0))

print('Results on Test data')
print(confusion_matrix(y_test,predict_test_resampled))
print(classification_report(y_test,predict_test_resampled,zero_division=0))

Results on Train data
[[43029 19291]
 [14911 47409]]
              precision    recall  f1-score   support

           0       0.74      0.69      0.72     62320
           1       0.71      0.76      0.73     62320

    accuracy                           0.73    124640
   macro avg       0.73      0.73      0.73    124640
weighted avg       0.73      0.73      0.73    124640

Results on Test data
[[18600  8060]
 [  244   651]]
              precision    recall  f1-score   support

           0       0.99      0.70      0.82     26660
           1       0.07      0.73      0.14       895

    accuracy                           0.70     27555
   macro avg       0.53      0.71      0.48     27555
weighted avg       0.96      0.70      0.80     27555



## StandardScaler and SMOTE

In [46]:
# Resampling
resampler = SMOTE(random_state=random_state)
X_train_resampled, y_train_resampled= resampler.fit_resample(X_train, y_train)

# Fitting
mlp_local = mlp.fit(X_train_resampled,y_train_resampled)

#Predicting
predict_train_resampled = mlp_local.predict(X_train_resampled)
predict_test_resampled = mlp_local.predict(X_test)

#Evaluating
print('Results on Train data')
print(confusion_matrix(y_train_resampled,predict_train_resampled))
print(classification_report(y_train_resampled,predict_train_resampled,zero_division=0))

print('Results on Test data')
print(confusion_matrix(y_test,predict_test_resampled))
print(classification_report(y_test,predict_test_resampled,zero_division=0))

Results on Train data
[[42743 19577]
 [15218 47102]]
              precision    recall  f1-score   support

           0       0.74      0.69      0.71     62320
           1       0.71      0.76      0.73     62320

    accuracy                           0.72    124640
   macro avg       0.72      0.72      0.72    124640
weighted avg       0.72      0.72      0.72    124640

Results on Test data
[[18438  8222]
 [  229   666]]
              precision    recall  f1-score   support

           0       0.99      0.69      0.81     26660
           1       0.07      0.74      0.14       895

    accuracy                           0.69     27555
   macro avg       0.53      0.72      0.47     27555
weighted avg       0.96      0.69      0.79     27555



## MinMaxScaler and RandomUnderSamping

In [47]:
# Resampling
resampler = RandomUnderSampler(random_state=random_state,sampling_strategy=0.95)
X_train_resampled, y_train_resampled= resampler.fit_resample(X_train, y_train)

# Fitting
mlp_local = mlp.fit(X_train_resampled,y_train_resampled)

#Predicting
predict_train_resampled = mlp_local.predict(X_train_resampled)
predict_test_resampled = mlp_local.predict(X_test)

#Evaluating
print('Results on Train data')
print(confusion_matrix(y_train_resampled,predict_train_resampled))
print(classification_report(y_train_resampled,predict_train_resampled,zero_division=0))

print('Results on Test data')
print(confusion_matrix(y_test,predict_test_resampled))
print(classification_report(y_test,predict_test_resampled,zero_division=0))

Results on Train data
[[1429  647]
 [ 478 1495]]
              precision    recall  f1-score   support

           0       0.75      0.69      0.72      2076
           1       0.70      0.76      0.73      1973

    accuracy                           0.72      4049
   macro avg       0.72      0.72      0.72      4049
weighted avg       0.72      0.72      0.72      4049

Results on Test data
[[18352  8308]
 [  224   671]]
              precision    recall  f1-score   support

           0       0.99      0.69      0.81     26660
           1       0.07      0.75      0.14       895

    accuracy                           0.69     27555
   macro avg       0.53      0.72      0.47     27555
weighted avg       0.96      0.69      0.79     27555



## MinMaxScaler and NearMissKNN

In [49]:
# Resampling
resampler = NearMiss(version=3)
X_train_resampled, y_train_resampled= resampler.fit_resample(X_train, y_train)

# Fitting
mlp_local = mlp.fit(X_train_resampled,y_train_resampled)

#Predicting
predict_train_resampled = mlp_local.predict(X_train_resampled)
predict_test_resampled = mlp_local.predict(X_test)

#Evaluating
print('Results on Train data')
print(confusion_matrix(y_train_resampled,predict_train_resampled))
print(classification_report(y_train_resampled,predict_train_resampled,zero_division=0))

print('Results on Test data')
print(confusion_matrix(y_test,predict_test_resampled))
print(classification_report(y_test,predict_test_resampled,zero_division=0))

Results on Train data
[[1660  313]
 [ 736 1237]]
              precision    recall  f1-score   support

           0       0.69      0.84      0.76      1973
           1       0.80      0.63      0.70      1973

    accuracy                           0.73      3946
   macro avg       0.75      0.73      0.73      3946
weighted avg       0.75      0.73      0.73      3946

Results on Test data
[[19123  7537]
 [  323   572]]
              precision    recall  f1-score   support

           0       0.98      0.72      0.83     26660
           1       0.07      0.64      0.13       895

    accuracy                           0.71     27555
   macro avg       0.53      0.68      0.48     27555
weighted avg       0.95      0.71      0.81     27555



# Cross Valiation with Stratified Cross Fold

Implementation reference:
#https://www.geeksforgeeks.org/stratified-k-fold-cross-validation/

In [61]:
# Create Resampling object
resampler = SMOTE(random_state=random_state)

# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)

# Crate Lists to store scores
lst_accu_stratified = []
lst_cr_stratified = []
lst_cm_stratified = []
lst_cr_precision_stratified= []
lst_cr_recall_stratified=[]
lst_cr_f1_stratified=[]

fold_no = 0
for train_index, test_index in skf.split(X,y):
    # K Fold Spliting
    print(fold_no,'K Fold Spliting')
    X_train_fold, X_test_fold = X_scaled[train_index], X_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    
    # Fitting
    print(fold_no,'K Fold Fitting')
    X_train_fold,y_train_fold = resampler.fit_resample(X_train_fold, y_train_fold)
    mlp_local.fit(X_train_fold, y_train_fold)
    
    
    #Predicting
    print(fold_no,'K Fold Predicting')
    y_pred_fold=mlp_local.predict(X_test_fold)
    
    #Scoring
    print(fold_no,'K Fold Scoring')
    ## accuracy
    accuracy = mlp_local.score(X_test_fold, y_test_fold)
    lst_accu_stratified.append(accuracy)
    
    ##confusiuon matrix
    cm = confusion_matrix(y_test_fold,y_pred_fold)
    lst_cm_stratified.append(cm)
    
    ##classification report
    cr = classification_report(y_test_fold,y_pred_fold,zero_division=0)
    lst_cr_stratified.append(cm)
    
    ##precision, recall, f1, support
    precision, recall, fbeta_score, support = precision_recall_fscore_support(y_test_fold, y_pred_fold)
    lst_cr_precision_stratified.append(precision[1])
    lst_cr_recall_stratified.append(recall[1])
    lst_cr_f1_stratified.append(fbeta_score[1])  
    
    #Printing
    print('Fold',str(fold_no))
    print(cr)
    print(cm)
    
    # K Fold Counter
    fold_no +=1




0 K Fold Spliting
0 K Fold Fitting
0 K Fold Predicting
0 K Fold Scoring
Fold 0
              precision    recall  f1-score   support

           0       0.99      0.73      0.84      8898
           1       0.08      0.69      0.14       287

    accuracy                           0.73      9185
   macro avg       0.53      0.71      0.49      9185
weighted avg       0.96      0.73      0.82      9185

[[6524 2374]
 [  88  199]]
1 K Fold Spliting
1 K Fold Fitting
1 K Fold Predicting
1 K Fold Scoring
Fold 1
              precision    recall  f1-score   support

           0       0.99      0.71      0.82      8898
           1       0.08      0.76      0.14       287

    accuracy                           0.71      9185
   macro avg       0.53      0.73      0.48      9185
weighted avg       0.96      0.71      0.80      9185

[[6290 2608]
 [  70  217]]
2 K Fold Spliting
2 K Fold Fitting
2 K Fold Predicting
2 K Fold Scoring
Fold 2
              precision    recall  f1-score   support



In [65]:
# Print the output.
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this mlp_local is:',
max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
mean(lst_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))

# Print the output.
print('List of possible precisions:', lst_cr_precision_stratified)
print('\nMaximum precisions That can be obtained from this mlp_local is:',
max(lst_cr_precision_stratified)*100, '%')
print('\nMinimum precisions:',
min(lst_cr_precision_stratified)*100, '%')
print('\nOverall precisions:',
mean(lst_cr_precision_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_cr_precision_stratified))

# Print the output.
print('List of possible recalls:', lst_cr_recall_stratified)
print('\nMaximum recalls That can be obtained from this mlp_local is:',
max(lst_cr_recall_stratified)*100, '%')
print('\nMinimum recalls:',
min(lst_cr_recall_stratified)*100, '%')
print('\nOverall recalls:',
mean(lst_cr_recall_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_cr_recall_stratified))

List of possible accuracy: [0.7319542732716385, 0.7084376701143168, 0.7630919978225368, 0.7316276537833424, 0.707348938486663, 0.7125748502994012, 0.7072400653238976, 0.7348938486663038, 0.7103658536585366, 0.7286585365853658]

Maximum Accuracy That can be obtained from this mlp_local is: 76.30919978225367 %

Minimum Accuracy: 70.72400653238977 %

Overall Accuracy: 72.36193688012003 %

Standard Deviation is: 0.017967032536604678
List of possible precisions: [0.07734162456276719, 0.0768141592920354, 0.08078118064802485, 0.07527301092043681, 0.07109682029296177, 0.07846649946255822, 0.07531824611032532, 0.07882352941176471, 0.07363505747126436, 0.07739463601532567]

Maximum precisions That can be obtained from this mlp_local is: 8.078118064802485 %

Minimum precisions: 7.109682029296177 %

Overall precisions: 7.6494476418746435 %

Standard Deviation is: 0.002779004705267312
List of possible recalls: [0.6933797909407665, 0.7560975609756098, 0.6341463414634146, 0.6724738675958188, 0.693379